In [1]:
import pandas as pd 
df = pd.read_csv("unicode-char-categories.csv")
# From https://www.fileformat.info/info/unicode/category/index.htm

In [2]:
from transformers import AutoTokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained("allenai/ivila-row-layoutlm-finetuned-s2vl-v2")

In [4]:
UNICODE_CATEGORIES_TO_REPLACE = ["Cc", "Cf", "Co", "Cs", "Mn", "Zl", "Zp", "Zs"]

In [5]:
def calculate_tokenization_len(row):    
    uni_code = r'\u' + row['Character'][2:]
    s = uni_code.encode().decode('unicode_escape')
    return len(tokenizer(s, add_special_tokens=False)['input_ids'])

In [6]:
df['Tokenization Length'] = df.apply(calculate_tokenization_len, axis=1)

In [7]:
df.groupby('Code')['Tokenization Length'].agg(['count', 'mean']).loc[UNICODE_CATEGORIES_TO_REPLACE]

,count,mean
Code,,
Cc,65,0.000000
Cf,163,0.766871
Co,6,0.666667
Cs,6,0.000000
Mn,1950,0.549744
Zl,1,0.000000
Zp,1,0.000000
Zs,17,0.000000


In [8]:
df[df['Tokenization Length']==0].Code.unique()

array(['Cc', 'Cf', 'Co', 'Cs', 'Mc', 'Mn', 'So', 'Zl', 'Zp', 'Zs'],
      dtype=object)

In [9]:
df[df['Tokenization Length']==0]['Character'].to_csv("zero-length-unicode-chars.txt", index=None, header=None)